In [1]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt, pow
from datetime import datetime
from time import sleep

In [2]:
df = pd.read_csv('datasets/c2ds1-2sp.txt', sep='\t')
df.head()

,sample_label,d1,d2
0,c2sp1s1,10.500000,9.000000
1,c2sp1s2,10.567170,9.268445
2,c2sp1s3,8.275320,11.382210
3,c2sp1s4,8.227458,11.377640
4,c2sp1s5,8.179511,11.372110


In [3]:
df3 = pd.read_csv('datasets/monkey.txt', sep='\t')
df3.columns = ['identificador','d1', 'd2']
df3.head()

,identificador,d1,d2
0,monkeyc1g1s1,8.809783,7.611147
1,monkeyc1g1s2,4.110747,11.103186
2,monkeyc1g1s3,4.114710,11.039587
3,monkeyc1g1s4,3.154736,6.743244
4,monkeyc1g1s5,5.972931,7.537982


# Algoritmo

In [4]:
#Distancia Euclidiana
# Mudei a forma como é calculado para poder utilizar vetorização
def distancia_euclidiana(x1, y1, x2, y2):
    X = x1 - x2
    Y = y1 - y2
    
    X = X*X
    Y = Y*Y
    
    distancia = np.sqrt(X+Y)
    return distancia

In [5]:
def calcular_distancia_full_optimized(df):
    lista = []
    for i, row in df.iterrows():
        distancias = distancia_euclidiana(row['d1'], row['d2'], df[i+1:]['d1'].values, df[i+1:]['d2'].values)
        
        lista.append(distancias)
      
    return lista

In [6]:
def dist_matrix(df):
    lista = []
    for i, row in df.iterrows():
        distancias = distancia_euclidiana(row['d1'], row['d2'], df['d1'].values, df['d2'].values)

        lista.append(distancias)
    
    matrix = np.matrix(lista)
    np.fill_diagonal(matrix, float('inf'))
    return matrix

In [7]:
def agrupar_clusters(clusters, objetos):
    obj1, obj2 =  objetos
    cluster1 = clusters[obj1]
    cluster2 = clusters[obj2]
    
    if cluster1 == cluster2:
        return False
    
    if cluster1 < cluster2:
        menor = cluster1
        maior = cluster2
    else:
        menor = cluster2
        maior = cluster1
        
    clusters[clusters == maior] = menor
    return True

In [8]:
def single_link(dataset, k_min, k_max, nome):
    numero_de_objetos = len(dataset)
    matrix = dist_matrix(dataset)
    clusters = np.arange(numero_de_objetos)
    
    qtd_clusters = numero_de_objetos
    while qtd_clusters > k_min:
        objetos = matrix.argmin()
        i = objetos // numero_de_objetos
        j = objetos % numero_de_objetos
        matrix[i,j] = float('inf')
        matrix[j,i] = float('inf')
        
        if agrupar_clusters(clusters, [i,j]):
            qtd_clusters -= 1
    
    return clusters

# Testes 

In [9]:
df_menor = df[:10].copy()
df_menor

,sample_label,d1,d2
0,c2sp1s1,10.500000,9.000000
1,c2sp1s2,10.567170,9.268445
2,c2sp1s3,8.275320,11.382210
3,c2sp1s4,8.227458,11.377640
4,c2sp1s5,8.179511,11.372110
5,c2sp1s6,8.131500,11.365610
6,c2sp1s7,8.083443,11.358140
7,c2sp1s8,8.035361,11.349700
8,c2sp1s9,7.987270,11.340270
9,c2sp1s10,7.939200,11.329870


In [10]:
clusters = single_link(df_menor, 2, 5, 'teste')

In [11]:
clusters

array([0, 0, 2, 2, 2, 2, 2, 2, 2, 2])

## Difrença entre `calcular_distancia_full_optimized` x  `dist_matrix`
Testes sobre a diferença de tempo entre calcular a matriz completa e apenas metade dela

In [13]:
%%timeit
teste = dist_matrix(df3)


451 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit

teste = calcular_distancia_full_optimized(df3)

1.14 s ± 48.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


O ganho de calcular a matriz inteira não é muito relevante, mas temos que considerar que ao calcular a matriz inteira ganhamos um leque muito grande de funções do ```numpy``` que devem ser adicionados as contas ao alterar a implementeção do algoritmo em si

## Colocando inf nas distâncias entre os mesmos clusters

Eu acho que a parte mais está demorando no momento é quando ele pega distâncias que pertencem ao mesmo cluster e com isso tem que realizar várias iterações até achar uma que funcione. Por isso o que vou tentar fazer é já colocar `float('inf')` nas distâncias de mesmo cluster quando ele juntar.

In [ ]:
2-1
2-1
10-1

In [82]:
matriz = dist_matrix(df_menor[:5])
clusters = np.arange(10)

matriz

matrix([[       inf, 0.27672103, 3.25946707, 3.28901492, 3.31836934],
        [0.27672103,        inf, 3.11778429, 3.15007235, 3.18218823],
        [3.25946707, 3.11778429,        inf, 0.04807968, 0.09633989],
        [3.28901492, 3.15007235, 0.04807968,        inf, 0.04826485],
        [3.31836934, 3.18218823, 0.09633989, 0.04826485,        inf]])

In [18]:
lista = [[0,0], [1,5], [5,1]]

for i,j in lista:
    print(matriz[i,j])

inf
3.214123424220825
3.214123424220825


In [ ]:
vetor = np.arange(10)
vetor

In [78]:
np.put(matriz, [[0,5,2]], [40])

In [79]:
matriz

matrix([[40.        ,  0.27672103, 40.        ,  3.28901492,  3.31836934],
        [40.        ,         inf,  3.11778429,  3.15007235,  3.18218823],
        [ 3.25946707,  3.11778429,         inf,  0.04807968,  0.09633989],
        [ 3.28901492,  3.15007235,  0.04807968,         inf,  0.04826485],
        [ 3.31836934,  3.18218823,  0.09633989,  0.04826485,         inf]])

In [65]:
vet1 = np.array([1,2,3,4])
vet2 = np.array([0,0,5,3])

def get_pos(u, v, num_obj):
    return u*num_obj + v


teste = get_pos(vet1, vet2, 5)
(li)

array([ 5, 10, 20, 23])

In [83]:
np.put(matriz, teste, [58])


In [84]:
matriz

matrix([[           inf, 2.76721027e-01, 3.25946707e+00, 3.28901492e+00,
         3.31836934e+00],
        [5.80000000e+01,            inf, 3.11778429e+00, 3.15007235e+00,
         3.18218823e+00],
        [5.80000000e+01, 3.11778429e+00,            inf, 4.80796833e-02,
         9.63398904e-02],
        [3.28901492e+00, 3.15007235e+00, 4.80796833e-02,            inf,
         4.82648496e-02],
        [5.80000000e+01, 3.18218823e+00, 9.63398904e-02, 5.80000000e+01,
                    inf]])